In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
import requests
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
import os
from math import radians, sin, cos, sqrt, asin
from backend.global_variables import DATA_PATH

load_dotenv()


def haversine(lat1, lon1, lat2, lon2):
    """Calculate the great circle distance between two points on the Earth's surface."""
    R = 6371.0  # Earth's radius in kilometers
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    return R * c

In [106]:
villes_df = pd.read_csv(
    "/Users/ugo/Documents/MH2D_projets/michlougo/Ligue1/backend/data/calculated_travels/name-stade.csv"
)
localisation_stades_df = pd.read_csv(
    "/Users/ugo/Documents/MH2D_projets/michlougo/Ligue1/backend/data/calculated_travels/localisation_stade.csv",
    index_col=0,
)

villes_df = villes_df.merge(
    localisation_stades_df[["Stadium", "latitude", "longitude"]],
    on="Stadium",
    how="left",
)

In [112]:
API_TOKEN = "eccc953e-c1f7-45f4-a828-bc319440fd3a"
url = f"https://api.sncf.com/v1/coverage/sncf/places"
places_ville = []

for idx_row, row in villes_df.iterrows():
    query = row["Location"]
    if query == "Monaco":
        query = "Nice"  # Monaco is not working, not in France I guess?
    params = {"q": query}
    response = requests.get(url, params=params, auth=HTTPBasicAuth(API_TOKEN, ""))
    data = response.json()
    for place in [
        place for place in data["places"] if place["id"].startswith("stop_area:")
    ]:
        if f"({query})" in place["name"]:
            latitude = float(place["stop_area"]["coord"]["lat"])
            longitude = float(place["stop_area"]["coord"]["lon"])
            places_ville.append(
                {
                    "location": row["Location"],
                    "team_name": row["Team"],
                    "stadium_name": row["Stadium"],
                    "stop_area_id": place["id"],
                    "gare_name": place["name"],
                    "latitude": latitude,
                    "longitude": longitude,
                    "distance_to_stadium": haversine(
                        float(row["latitude"]),
                        float(row["longitude"]),
                        latitude,
                        longitude,
                    ),
                }
            )

In [113]:
pd.DataFrame(places_ville).to_csv(DATA_PATH + "gare_positions.csv", index=False)

## Get cached gares for streamlit plots

In [8]:
train_trajets_df = pd.read_csv(DATA_PATH + "train_emissions.csv")
gare_locations_foot = pd.read_csv(DATA_PATH + "gare_positions.csv")

In [81]:
cache_gares = []
for idx_row, row in gare_locations_foot.iterrows():
    cache_gares.append(
        {
            "base_name": row["gare_name"],
            "stop_area_id": row["stop_area_id"],
            "gare_name": row["gare_name"],
            "latitude": row["latitude"],
            "longitude": row["longitude"],
        }
    )
print(len(cache_gares))

71


In [ ]:
import ast

unique_stations = set[str]()
for ex_route_detail in train_trajets_df["route_details"].to_list():
    ex_route_detail = ast.literal_eval(ex_route_detail)
    for segment in ex_route_detail["train_route_details"]:
        unique_stations.add(segment["from"])
        unique_stations.add(segment["to"])

In [83]:
# Make a set of gare_names already in the cache
cache_gare_names = set([gare["base_name"] for gare in cache_gares])
API_TOKEN = "eccc953e-c1f7-45f4-a828-bc319440fd3a"
url = f"https://api.sncf.com/v1/coverage/sncf/places"

for unique_station in unique_stations:
    if unique_station not in cache_gare_names:
        # Example: fetch the station details by name - you need to implement 'get_gare_info'
        params = {"q": unique_station}
        response = requests.get(url, params=params, auth=HTTPBasicAuth(API_TOKEN, ""))
        try:
            data = response.json()
            places = [
                place
                for place in data["places"]
                if place["id"].startswith("stop_area:")
            ]
            closest_place = places[0]
            cache_gares.append(
                {
                    "base_name": unique_station,
                    "stop_area_id": closest_place["id"],
                    "gare_name": closest_place["name"],
                    "latitude": closest_place["stop_area"]["coord"]["lat"],
                    "longitude": closest_place["stop_area"]["coord"]["lon"],
                }
            )
            cache_gare_names.add(unique_station)

        except Exception as e:
            print(f"Error fetching {unique_station}: {e}")
            continue

Error fetching Haussmann - Saint-Lazare (Paris): 'places'
Error fetching Paris - Gare de Lyon - Banlieue (Paris): 'places'
Error fetching Saint-Michel Notre-Dame (Paris): 'places'


In [ ]:
manual_adds = [
    {
        "stop_area_id": "unknown_saintlaz",
        "gare_name": "Haussmann - Saint-Lazare (Paris)",
        "latitude": 48.87595523999466,
        "longitude": 2.3289166381708086,
    },
    {
        "stop_area_id": "unknown_garelyon",
        "gare_name": "Paris - Gare de Lyon - Banlieue (Paris)",
        "latitude": 48.84437592875396,
        "longitude": 2.3735323285829764,
    },
    {
        "stop_area_id": "unknown_saintmichel",
        "gare_name": "Saint-Michel Notre-Dame (Paris)",
        "latitude": 48.85363647995824,
        "longitude": 2.3445081974287514,
    },
]

pd.DataFrame(cache_gares).to_csv(
    DATA_PATH + "general_gare_position_cache.csv", index=False
)

## test sncf angers lens

In [39]:
from backend.services.train_service import TrainTrajetService
import os
from datetime import datetime, timedelta
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

train_service = TrainTrajetService(
    api_key=os.getenv("GOOGLE_MAPS_API_KEY"),
    sncf_api_key=os.getenv("SNCF_API_KEY"),
)

[21:07:04] INFO     Loaded 223 cached road distances                                  ]8;id=807006;file:///Users/ugo/Documents/MH2D_projets/michlougo/Ligue1/backend/services/base_transport_service.py\base_transport_service.py]8;;\:]8;id=629889;file:///Users/ugo/Documents/MH2D_projets/michlougo/Ligue1/backend/services/base_transport_service.py#150\150]8;;\

           INFO     Loaded 71 gare positions from cache                                         ]8;id=62954;file:///Users/ugo/Documents/MH2D_projets/michlougo/Ligue1/backend/services/train_service.py\train_service.py]8;;\:]8;id=180122;file:///Users/ugo/Documents/MH2D_projets/michlougo/Ligue1/backend/services/train_service.py#62\62]8;;\

           INFO     Loaded 223 cached road distances                                  ]8;id=693805;file:///Users/ugo/Documents/MH2D_projets/michlougo/Ligue1/backend/services/base_transport_service.py\base_transport_service.py]8;;\:]8;id=588336;file:///Users/ugo/Documents/MH2D_projets/michlougo/Ligue1/backend/services/base_transport_service.py#150\150]8;;\

In [4]:
cache_gares = pd.read_csv(DATA_PATH + "gare_positions.csv")

In [5]:
cache_gares[cache_gares["team_name"] == "Lens"]

,location,team_name,stadium_name,stop_area_id,gare_name,latitude,longitude,distance_to_stadium
5,Lens,Lens,Stade Bollaert-Delelis,stop_area:SNCF:87345025,Lens (Lens),50.426866,2.828301,1.150465
6,Lens,Lens,Stade Bollaert-Delelis,stop_area:SNCF:87345033,Loos-en-Gohelle (Lens),50.437864,2.793543,1.623204


In [ ]:
departure = "Angers"
arrival = "Lens"
stop_area_departures: list[str] = train_service.gare_positions_df[
    train_service.gare_positions_df["team_name"] == departure
]["stop_area_id"].tolist()
stop_area_arrivals: list[str] = train_service.gare_positions_df[
    train_service.gare_positions_df["team_name"] == arrival
]["stop_area_id"].tolist()
train_service.logger.debug("departure: %s", departure)
train_service.logger.debug("stop_area_departures: %s", stop_area_departures)
train_service.logger.debug("arrival: %s", arrival)
train_service.logger.debug("stop_area_arrivals: %s", stop_area_arrivals)

week_trains = []
for stop_area_departure in tqdm(stop_area_departures[:1]):
    for stop_area_arrival in stop_area_arrivals[:1]:
        new_week_trains = train_service._get_sncf_journeys(
            stop_area_departure,
            stop_area_arrival,
            datetime.now().replace(hour=7, minute=0, second=0, microsecond=0)
            + timedelta(days=2),
            3,
        )
        new_week_trains = [
            {
                "stop_area_departure_id": stop_area_departure,
                "stop_area_arrival_id": stop_area_arrival,
            }
            | week_train
            for week_train in new_week_trains
        ]
        week_trains.extend(new_week_trains)
# # train part
# fastest_train = min(
#     week_trains,
#     key=lambda train: train_service._trip_stats(train["sections"])["duration_s"],
# )
# fastest_train_route = train_service._trip_stats(fastest_train["sections"])

[20:30:54] DEBUG    departure: Angers                                                               ]8;id=973269;file:///var/folders/j3/mm5hxwf11p18skbfc4mkjsgr0000gn/T/ipykernel_5996/1403101462.py\1403101462.py]8;;\:]8;id=135544;file:///var/folders/j3/mm5hxwf11p18skbfc4mkjsgr0000gn/T/ipykernel_5996/1403101462.py#9\9]8;;\

           DEBUG    stop_area_departures: ['stop_area:SNCF:87484006', 'stop_area:SNCF:87484048']   ]8;id=815133;file:///var/folders/j3/mm5hxwf11p18skbfc4mkjsgr0000gn/T/ipykernel_5996/1403101462.py\1403101462.py]8;;\:]8;id=233364;file:///var/folders/j3/mm5hxwf11p18skbfc4mkjsgr0000gn/T/ipykernel_5996/1403101462.py#10\10]8;;\

           DEBUG    arrival: Lens                                                                  ]8;id=233393;file:///var/folders/j3/mm5hxwf11p18skbfc4mkjsgr0000gn/T/ipykernel_5996/1403101462.py\1403101462.py]8;;\:]8;id=618829;file:///var/folders/j3/mm5hxwf11p18skbfc4mkjsgr0000gn/T/ipykernel_5996/1403101462.py#11\11]8;;\

           DEBUG    stop_area_arrivals: ['stop_area:SNCF:87345025', 'stop_area:SNCF:87345033']     ]8;id=660519;file:///var/folders/j3/mm5hxwf11p18skbfc4mkjsgr0000gn/T/ipykernel_5996/1403101462.py\1403101462.py]8;;\:]8;id=198211;file:///var/folders/j3/mm5hxwf11p18skbfc4mkjsgr0000gn/T/ipykernel_5996/1403101462.py#12\12]8;;\

  0%|          | 0/2 [00:00<?, ?it/s]

...

           DEBUG    Starting new HTTPS connection (1): api.sncf.com:443                      ]8;id=395691;file:///Users/ugo/miniconda3/envs/aqemia/lib/python3.11/site-packages/urllib3/connectionpool.py\connectionpool.py]8;;\:]8;id=278654;file:///Users/ugo/miniconda3/envs/aqemia/lib/python3.11/site-packages/urllib3/connectionpool.py#1049\1049]8;;\

[20:31:06] DEBUG    https://api.sncf.com:443 "GET                                             ]8;id=629466;file:///Users/ugo/miniconda3/envs/aqemia/lib/python3.11/site-packages/urllib3/connectionpool.py\connectionpool.py]8;;\:]8;id=213761;file:///Users/ugo/miniconda3/envs/aqemia/lib/python3.11/site-packages/urllib3/connectionpool.py#544\544]8;;\
                    /v1/coverage/sncf/journeys?from=stop_area%3ASNCF%3A87484048&to=stop_area%                      
                    3ASNCF%3A87345033&datetime=20251110T070000 HTTP/1.1" 200 18009                                 

           DEBUG    Starting new HTTPS connection (1): api.sncf.com:443                      ]8;id=468444;file:///Users/ugo/miniconda3/envs/aqemia/lib/python3.11/site-packages/urllib3/connectionpool.py\connectionpool.py]8;;\:]8;id=819337;file:///Users/ugo/miniconda3/envs/aqemia/lib/python3.11/site-packages/urllib3/connectionpool.py#1049\1049]8;;\

           DEBUG    https://api.sncf.com:443 "GET                                             ]8;id=988933;file:///Users/ugo/miniconda3/envs/aqemia/lib/python3.11/site-packages/urllib3/connectionpool.py\connectionpool.py]8;;\:]8;id=387823;file:///Users/ugo/miniconda3/envs/aqemia/lib/python3.11/site-packages/urllib3/connectionpool.py#544\544]8;;\
                    /v1/coverage/sncf/journeys?from=stop_area%3ASNCF%3A87484048&to=stop_area%                      
                    3ASNCF%3A87345033&datetime=20251111T070000 HTTP/1.1" 200 186                                   

100%|██████████| 2/2 [00:11<00:00,  5.69s/it]


In [19]:
types_section = set[str]()
commercial_modes = set[str]()
for wk_train in week_trains:
    for section in wk_train["sections"]:
        types_section.add(section["type"])
        commercial_modes.add(
            section.get("display_informations", {}).get("physical_mode")
        )
        if section["type"] == "waiting":
            print(section)

{'id': 'section_2_0', 'duration': 420, 'co2_emission': {'value': 0.0, 'unit': 'gEC'}, 'departure_date_time': '20251109T091300', 'arrival_date_time': '20251109T092000', 'type': 'waiting', 'links': []}
{'id': 'section_4_0', 'duration': 1740, 'co2_emission': {'value': 0.0, 'unit': 'gEC'}, 'departure_date_time': '20251109T094600', 'arrival_date_time': '20251109T101500', 'type': 'waiting', 'links': []}
{'id': 'section_7_0', 'duration': 620, 'co2_emission': {'value': 0.0, 'unit': 'gEC'}, 'departure_date_time': '20251109T103940', 'arrival_date_time': '20251109T105000', 'type': 'waiting', 'links': []}
{'id': 'section_10_0', 'duration': 601, 'co2_emission': {'value': 0.0, 'unit': 'gEC'}, 'departure_date_time': '20251109T113059', 'arrival_date_time': '20251109T114100', 'type': 'waiting', 'links': []}
{'id': 'section_13_0', 'duration': 1560, 'co2_emission': {'value': 0.0, 'unit': 'gEC'}, 'departure_date_time': '20251109T125300', 'arrival_date_time': '20251109T131900', 'type': 'waiting', 'links': 

In [14]:
types_section

{'boarding', 'crow_fly', 'public_transport', 'transfer', 'waiting'}

In [12]:
for section in week_trains[0]["sections"]:
    print("========")
    print("type_section:", section["type"])
    try:
        print("FROM:")
        print(section["from"]["id"])
        print(section["from"]["name"])
        print("TO:")
        print(section["to"]["id"])
        print(section["to"]["name"])
        print(section.get("display_informations", {}))
        # print(section)
    except:
        pass
        # print(section)


type_section: crow_fly
FROM:
stop_area:SNCF:87484006
Angers Saint-Laud (Angers)
TO:
stop_point:SNCF:87484006:LongDistanceTrain
Angers Saint-Laud (Angers)
{}
type_section: public_transport
FROM:
stop_point:SNCF:87484006:LongDistanceTrain
Angers Saint-Laud (Angers)
TO:
stop_point:SNCF:87391003:LongDistanceTrain
Paris - Montparnasse - Hall 1 & 2 (Paris)
{'commercial_mode': 'TGV INOUI', 'network': 'TGV INOUI', 'direction': 'Paris - Montparnasse - Hall 1 & 2 (Paris)', 'label': 'Nantes - Paris - Montparnasse - Hall 1 & 2', 'color': '', 'code': '', 'headsign': '8804', 'name': 'Nantes - Paris - Montparnasse - Hall 1 & 2', 'links': [{'templated': False, 'rel': 'terminus', 'internal': True, 'type': 'stop_area', 'id': 'stop_area:SNCF:87391003'}], 'text_color': '', 'trip_short_name': '8804', 'company': 'SNCF Voyageurs', 'description': '', 'physical_mode': 'Train grande vitesse', 'equipments': []}
type_section: transfer
FROM:
stop_point:SNCF:87391003:LongDistanceTrain
Paris - Montparnasse - Hall 1 

In [43]:
output = train_service._trip_stats(
    week_trains[0]["sections"], compute_using_google=False
)

yes oui check
{'id': 'section_3_0', 'duration': 1560, 'co2_emission': {'value': 115.34432, 'unit': 'gEC'}, 'departure_date_time': '20251109T092000', 'arrival_date_time': '20251109T094600', 'base_departure_date_time': '20251109T092000', 'base_arrival_date_time': '20251109T094600', 'data_freshness': 'base_schedule', 'to': {'id': 'stop_point:SNCF:87393009:RapidTransit', 'name': 'Versailles Chantiers (Versailles)', 'quality': 0, 'stop_point': {'id': 'stop_point:SNCF:87393009:RapidTransit', 'name': 'Versailles Chantiers', 'label': 'Versailles Chantiers (Versailles)', 'coord': {'lon': '2.135883', 'lat': '48.795826'}, 'links': [], 'administrative_regions': [{'id': 'admin:fr:78646', 'name': 'Versailles', 'level': 8, 'zip_code': '78000', 'label': 'Versailles (78000)', 'insee': '78646', 'coord': {'lon': '2.1266886', 'lat': '48.80354029999999'}}], 'stop_area': {'id': 'stop_area:SNCF:87393009', 'name': 'Versailles Chantiers', 'codes': [{'type': 'source', 'value': '87393009'}, {'type': 'uic', 'valu

In [44]:
output

{'carbon_emission_kgCO2': 9.355926867026552,
 'distance_km': 505.39476124468433,
 'duration_s': 11776.182809617276,
 'details': [{'from': 'Angers Saint-Laud (Angers)',
   'to': 'Paris - Montparnasse - Hall 1 & 2 (Paris)',
   'type': 'public_transport',
   'distance_km': 262.698,
   'co2_kg': 0.61996728,
   'time_s': 5040},
  {'from': 'Paris - Montparnasse - Hall 1 & 2 (Paris)',
   'to': 'Gare du Nord (Paris)',
   'type': 'car',
   'distance_km': 5.224761244684369,
   'co2_kg': 7.837141867026552,
   'time_s': 376.18280961727453},
  {'from': 'Paris Nord (Paris)',
   'to': 'Lille Flandres (Lille)',
   'type': 'public_transport',
   'distance_km': 202.002,
   'co2_kg': 0.47672472,
   'time_s': 4020},
  {'from': 'Lille Flandres (Lille)',
   'to': 'Lens (Lens)',
   'type': 'public_transport',
   'distance_km': 35.47,
   'co2_kg': 0.422093,
   'time_s': 2340}]}

In [42]:
train_service.calculate_distance(48.841172, 2.320514, 48.881393, 2.357438)

48.841172 2.320514 48.881393 2.357438


5.224761244684369

In [45]:
output_2 = train_service._trip_stats(
    week_trains[0]["sections"], compute_using_google=True
)

yes oui check
{'id': 'section_3_0', 'duration': 1560, 'co2_emission': {'value': 115.34432, 'unit': 'gEC'}, 'departure_date_time': '20251109T092000', 'arrival_date_time': '20251109T094600', 'base_departure_date_time': '20251109T092000', 'base_arrival_date_time': '20251109T094600', 'data_freshness': 'base_schedule', 'to': {'id': 'stop_point:SNCF:87393009:RapidTransit', 'name': 'Versailles Chantiers (Versailles)', 'quality': 0, 'stop_point': {'id': 'stop_point:SNCF:87393009:RapidTransit', 'name': 'Versailles Chantiers', 'label': 'Versailles Chantiers (Versailles)', 'coord': {'lon': '2.135883', 'lat': '48.795826'}, 'links': [], 'administrative_regions': [{'id': 'admin:fr:78646', 'name': 'Versailles', 'level': 8, 'zip_code': '78000', 'label': 'Versailles (78000)', 'insee': '78646', 'coord': {'lon': '2.1266886', 'lat': '48.80354029999999'}}], 'stop_area': {'id': 'stop_area:SNCF:87393009', 'name': 'Versailles Chantiers', 'codes': [{'type': 'source', 'value': '87393009'}, {'type': 'uic', 'valu

[21:08:58] DEBUG    Making API request for road distance: 48.841172,2.320514 ->       ]8;id=304582;file:///Users/ugo/Documents/MH2D_projets/michlougo/Ligue1/backend/services/base_transport_service.py\base_transport_service.py]8;;\:]8;id=723850;file:///Users/ugo/Documents/MH2D_projets/michlougo/Ligue1/backend/services/base_transport_service.py#322\322]8;;\
                    48.881393,2.357438                                                                             

           DEBUG    Making Google Maps API request to:                                ]8;id=805355;file:///Users/ugo/Documents/MH2D_projets/michlougo/Ligue1/backend/services/base_transport_service.py\base_transport_service.py]8;;\:]8;id=516540;file:///Users/ugo/Documents/MH2D_projets/michlougo/Ligue1/backend/services/base_transport_service.py#111\111]8;;\
                    https://routes.googleapis.com/directions/v2:computeRoutes                                      

           DEBUG    Starting new HTTPS connection (1): routes.googleapis.com:443             ]8;id=555252;file:///Users/ugo/miniconda3/envs/aqemia/lib/python3.11/site-packages/urllib3/connectionpool.py\connectionpool.py]8;;\:]8;id=302442;file:///Users/ugo/miniconda3/envs/aqemia/lib/python3.11/site-packages/urllib3/connectionpool.py#1049\1049]8;;\

[21:09:02] DEBUG    https://routes.googleapis.com:443 "POST /directions/v2:computeRoutes      ]8;id=210775;file:///Users/ugo/miniconda3/envs/aqemia/lib/python3.11/site-packages/urllib3/connectionpool.py\connectionpool.py]8;;\:]8;id=840224;file:///Users/ugo/miniconda3/envs/aqemia/lib/python3.11/site-packages/urllib3/connectionpool.py#544\544]8;;\
                    HTTP/1.1" 200 None                                                                             

           INFO     Saved 224 road distances to cache                                 ]8;id=287330;file:///Users/ugo/Documents/MH2D_projets/michlougo/Ligue1/backend/services/base_transport_service.py\base_transport_service.py]8;;\:]8;id=911749;file:///Users/ugo/Documents/MH2D_projets/michlougo/Ligue1/backend/services/base_transport_service.py#190\190]8;;\

4


In [46]:
output_2

{'carbon_emission_kgCO2': 30.450784999999996,
 'distance_km': 519.458,
 'duration_s': 16004,
 'details': [{'from': 'Angers Saint-Laud (Angers)',
   'to': 'Paris - Montparnasse - Hall 1 & 2 (Paris)',
   'type': 'public_transport',
   'distance_km': 262.698,
   'co2_kg': 0.61996728,
   'time_s': 5040},
  {'from': 'Paris - Montparnasse - Hall 1 & 2 (Paris)',
   'to': 'Gare du Nord (Paris)',
   'type': 'car',
   'distance_km': 19.288,
   'co2_kg': 28.931999999999995,
   'time_s': 4604},
  {'from': 'Paris Nord (Paris)',
   'to': 'Lille Flandres (Lille)',
   'type': 'public_transport',
   'distance_km': 202.002,
   'co2_kg': 0.47672472,
   'time_s': 4020},
  {'from': 'Lille Flandres (Lille)',
   'to': 'Lens (Lens)',
   'type': 'public_transport',
   'distance_km': 35.47,
   'co2_kg': 0.422093,
   'time_s': 2340}]}